In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#!unzip /content/gdrive/MyDrive/IMP/embedded_data-20210323T034408Z-001.zip -d /content/gdrive/MyDrive/IMP/e

In [ ]:
import json
import numpy as np

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Concatenate,LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [ ]:
filetry = '/content/gdrive/MyDrive/embedded_data/emd_im_name_feature.json'
with open(filetry) as train_file1:
    emd_im_name_feature = json.load(train_file1)
for k,v in emd_im_name_feature.items():
  print(k)
  print((np.array(v)).shape)

In [ ]:
filetry = '/content/gdrive/MyDrive/embedded_data/emd_im_name_article_list_tokenized.json'
with open(filetry) as train_file1:
    emd_im_name_article_list_tokenized = json.load(train_file1)
for k,v in emd_im_name_article_list_tokenized.items():
  print(k)
  print((np.array(v)).shape)

In [ ]:
filetry = '/content/gdrive/MyDrive/embedded_data/emd_im_name_caption_tokenized.json'
with open(filetry) as train_file1:
    emd_im_name_caption_tokenized = json.load(train_file1)
for k,v in emd_im_name_caption_tokenized.items():
  print(k)
  print((np.array(v)).shape)

In [ ]:
all_data=[]
for k,v in emd_im_name_feature.items():
  all_data.append(k)

In [ ]:
array=[]
for i in range(1500):
  array.append(np.array(emd_im_name_article_list_tokenized[all_data[i]]))
X_article_train=np.stack(array, axis=0)
print(X_article_train.shape)

(1500, 40, 1024)


In [ ]:
array=[]
for i in range(1500):
  npp=np.array(emd_im_name_caption_tokenized[all_data[i]])
  npp=npp.flatten()
  array.append(npp)
Y_caption_train=np.stack(array, axis=0)
print(Y_caption_train.shape)

(1500, 1024)


In [ ]:
array=[]
for i in range(1500):
  npp=np.array(emd_im_name_feature[all_data[i]])
  npp=npp.flatten()
  array.append(npp)

X_image_train=np.stack(array, axis=0)
print(X_image_train.shape)

(1500, 1024)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class PositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super(PositionEmbedding, self).__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions

In [ ]:
maxlen = 40
embed_dim = 2048  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

In [ ]:
#inputs = layers.Input(shape=(maxlen,))
#embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
#x = embedding_layer(inputs)
#transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
#x = transformer_block(x)
#x = layers.GlobalAveragePooling1D()(x)
#x = layers.Dropout(0.1)(x)
#x = layers.Dense(20, activation="relu")(x)
#x = layers.Dropout(0.1)(x)
#outputs = layers.Dense(2, activation="softmax")(x)

#model = keras.Model(inputs=inputs, outputs=outputs)

Model

In [ ]:
input1 = Input(shape=(40,1024))
inp= Dropout(0.1)(input1)
ls_art = LSTM(1024)(inp)
input2 = Input(shape=(1024,))
con_cat = Concatenate()([ls_art, input2])
re= Reshape((1, 2048), input_shape=(2048,))(con_cat)
re1= Dropout(0.2)(re)

embedding_layer = PositionEmbedding(maxlen, embed_dim)
re1 = embedding_layer(re1)

transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
ls =transformer_block(re1)
ls_dense = Dense(1024,activation='relu')(ls)
ls1 = layers.GlobalAveragePooling1D()(ls_dense)
model = Model(inputs=[input1, input2], outputs=ls1)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 40, 1024)]   0                                            
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 40, 1024)     0           input_3[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 1024)         8392704     dropout_4[0][0]                  
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1024)]       0                                            
____________________________________________________________________________________________

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.1,momentum=0.7)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=opt)

In [ ]:
model.fit(x=[X_article_train,X_image_train],y=Y_caption_train,epochs=30)

Testing

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from google.colab.patches import cv2_imshow

In [ ]:
#!unzip /content/gdrive/MyDrive/IMP/btp_data-20210323T040838Z-001.zip -d /content/gdrive/MyDrive/IMP/b

In [ ]:
filetry = '/content/gdrive/MyDrive/btp_data/im_name_article_list_1.json'
with open(filetry) as train_file1:
    article_list_real = json.load(train_file1)
filetry = '/content/gdrive/MyDrive/btp_data/im_name_caption.json'
with open(filetry) as train_file1:
    caption_real = json.load(train_file1)

In [ ]:
filetry = '/content/gdrive/MyDrive/btp_data/im_name_capno_50_final.json'
with open(filetry) as train_file1:
    img_cap_real = json.load(train_file1)

In [ ]:
array=[]
for i in range(1505,1600):
  array.append(np.array(emd_im_name_article_list_tokenized[all_data[i]]))
X_article_test=np.stack(array, axis=0)
print(X_article_test.shape)

(95, 40, 1024)


In [ ]:
array=[]
for i in range(1505,1600):
  npp=np.array(emd_im_name_feature[all_data[i]])
  npp=npp.flatten()
  array.append(npp)

X_image_test=np.stack(array, axis=0)
print(X_image_test.shape)

(95, 1024)


In [ ]:
output=model.predict([X_article_test,X_image_test])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


In [ ]:
import cv2

from nltk.translate.bleu_score import sentence_bleu

In [ ]:
for i in range(1512,1600):
  key=all_data[i]
  context_vec = output[i-1505]
  #print(context_vec.shape)
  m=-2
  cap_final=""
  for j in range(0,len(article_list_real[key])):
    npo = np.array(emd_im_name_article_list_tokenized[key]) 
    B = npo[j]
    cos_sim=cosine_similarity(context_vec.reshape(1,-1),B.reshape(1,-1))
    if cos_sim > m:
      m=cos_sim
      cap_final = article_list_real[key][j]
  img_path='/content/gdrive/MyDrive/Use Image/'+str(key)+'_'+str(img_cap_real[key]) +'.jpg'
  image = cv2.imread(img_path)
  print(cv2_imshow(image))
  print("Caption:")
  print(cap_final)
  print("True val")
  print(caption_real[key])
  print("\n\n")
